In [368]:
import numpy as np
import pandas as pd
import geopandas
import altair as alt
import pydeck as pdk
from sklearn.cluster import Birch
import matplotlib.pyplot as plt
from matplotlib import cm

In [469]:
with open("./charts/chart.json") as f:
    chart = f.open()
reg_chart = open("./charts/reg_chart.json")

In [473]:
import json
print(json.dumps(chart))

TypeError: Object of type TextIOWrapper is not JSON serializable

In [369]:
# geo_dpnk = geopandas.read_file('./data_project3/dpnk_json.geojson')
# geo_roads = geopandas.read_file('./data_project3/cykloopatreni_json.geojson')

In [444]:
cmap = plt.get_cmap('plasma', 12)
rgba_array = cm.ScalarMappable(cmap=cmap).to_rgba(list(range(12)), bytes=True)
rgba = [tuple(int(q) for q in x) for x in rgba_array]  # values to int
rgba

[(12, 7, 134, 255),
 (61, 3, 155, 255),
 (98, 0, 166, 255),
 (133, 6, 166, 255),
 (165, 31, 151, 255),
 (191, 57, 130, 255),
 (213, 83, 109, 255),
 (231, 110, 90, 255),
 (245, 141, 69, 255),
 (252, 173, 49, 255),
 (251, 209, 36, 255),
 (239, 248, 33, 255)]

In [457]:
np.linspace(1994,2022, 10).round()

array([1994., 1997., 2000., 2003., 2006., 2010., 2013., 2016., 2019.,
       2022.])

In [452]:
dff = pd.DataFrame()
dff['labels'] = range(15)
dff['number'] = 1
dff

,labels,number
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1


In [453]:
alt.Chart(dff).mark_rect().encode(x=alt.X('labels:O', axis=alt.Axis(values=[0,11], labelAngle=0)), y=alt.Y('number:O', axis=None), color=alt.Color("labels", scale=alt.Scale(scheme="plasma"), legend=None)).properties(height=50, width=350)

alt.Chart(...)

In [423]:
df = pd.DataFrame()
df['color'] = rgba
df['number'] = 1
df2 = pd.DataFrame({'a': [12], 'b': ['col']})
df['labels'] = range(len(df))
df.loc[0, 'labels'] = 0
df.loc[df.index[-1], 'labels'] = 11
df

,color,number,labels
0,"(12, 7, 134, 255)",1,0
1,"(61, 3, 155, 255)",1,1
2,"(98, 0, 166, 255)",1,2
3,"(133, 6, 166, 255)",1,3
4,"(165, 31, 151, 255)",1,4
5,"(191, 57, 130, 255)",1,5
6,"(213, 83, 109, 255)",1,6
7,"(231, 110, 90, 255)",1,7
8,"(245, 141, 69, 255)",1,8
9,"(252, 173, 49, 255)",1,9


In [438]:
chart = alt.Chart(df).mark_rect().encode(x=alt.X('labels:O', axis=alt.Axis(values=[0,11])), y=alt.Y('number:O', axis=None), color=alt.Color("labels", scale=alt.Scale(scheme="plasma"), legend=None)).properties(height=50, width=350)
chart

alt.Chart(...)

In [462]:
df2 = pd.DataFrame({
    'x': range(20),
    'y': range(20)
})

xx = alt.Chart(df2).mark_line().encode(
    alt.X('x', axis=alt.Axis(values=[0, 10, 15, 20])),
    alt.Y('y')
)
xx.save('test_chart.json')

In [374]:
df_short = df.iloc[:4].copy()
df_short['labels'] = [0, ' ', ' ', 888]
df_short['bruh'] = range(4)
df_short

,color,number,labels,bruh
0,"(12, 7, 134, 255)",1,0,0
1,"(61, 3, 155, 255)",1,,1
2,"(98, 0, 166, 255)",1,,2
3,"(133, 6, 166, 255)",1,888,3


In [392]:
alt.Chart(df_short).mark_rect().encode(
    x=alt.X('bruh:O', axis=alt.Axis(values=[0,3])), # values=[0,1,3] , tickCount=3, title='range')
    y=alt.Y('number:O', axis=None),
    color=alt.Color("color", scale=alt.Scale(scheme="plasma"), legend=None)).properties(height=50, width=350)


alt.Chart(...)

In [376]:
chart = alt.Chart(df2).mark_bar(size=20, color=alt.Gradient(gradient='linear',stops=[
                            alt.GradientStop(color='blue', offset=0),
                            alt.GradientStop(color='purple', offset=0.5),
                            alt.GradientStop(color='yellow', offset=1)
                        ])).encode(x='a:Q', y='a:O')
chart

alt.Chart(...)

In [377]:
# chart = alt.Chart(df2).mark_bar(size=20).encode(x='a:Q', y='a:O', color=alt.Color('a:Q', scale=alt.Scale(scheme='plasma')))
# chart = alt.Chart(df2).mark_bar(size=20, color=alt.Gradient(gradient='linear',stops=[alt.GradientStop(color=low, offset=0),alt.GradientStop(color=low, offset=1)])).encode(x='a:Q', y='a:O')
# chart = alt.Chart(df2).mark_bar(size=20, fill=low).encode(x='a:Q', y='a:O')
# chart = alt.Chart(df2).mark_bar(size=20).encode(x='a:Q', y='a:O', color=alt.Color('a:Q', scale=alt.Scale(scheme='plasma')), fill=alt.Gradient('color'))
# chart = alt.Chart(df2).mark_bar(size=20, color=alt.Gradient(gradient='linear',stops=[[0,0],[0,1]])).encode(x='a:Q', y='a:O')
# bins=list(range(12)) ,domainMin=0, domainMax=14

In [378]:
# layer2 = pdk.Layer(
#     type="GeoJsonLayer",
#     data=geo_test,
#     pickable=True,
#     auto_highlight=True,
#     get_line_color='color',
#     line_width_scale=20
# )
# initial_position = pdk.ViewState(latitude=49.196023157428, longitude=16.60988, zoom=11, pitch=0, bearing=0)
# deck = pdk.Deck(layers=[layer2], initial_view_state=initial_position, tooltip={"text": "Number of cyclists per mont {birch_cmap} (in thousands)"})
# # deck.to_html("path_layer.html")
# deck

In [379]:
# layer2 = pdk.Layer(
#     type="GeoJsonLayer",
#     data=geo_roads,
#     pickable=True,
#     # get_fill_color=[255, 255, 255],
#     get_line_color='color',
#     # get_width=100,
#     line_width_scale=20
# )
# initial_position = pdk.ViewState(latitude=49.196023157428, longitude=16.60988, zoom=11, pitch=0, bearing=0)
# deck = pdk.Deck(layers=[layer2], initial_view_state=initial_position, tooltip={"text": "Built in {rok_realizace}"})
# # deck.to_html("path_layer.html")
# deck

In [380]:
# layer = pdk.Layer(
#     'ScatterplotLayer',
#     data=geo_roads,
#     get_position=['lat', 'lon'],
#     # get_position="[lat, lon]",
#     auto_highlight=True,
#     get_radius=50,          # Radius is given in meters
#     get_fill_color=[180, 0, 200, 140],  # Set an RGBA value for fill
#     pickable=True)

# initial_position = pdk.ViewState(latitude=49.196023157428, longitude=16.60988, zoom=11, pitch=0, bearing=0)
# deck = pdk.Deck(layers=[layer], initial_view_state=initial_position)
# deck